In [1]:
import pandas as pd
import acquire
import prepare
import wrangle

In [2]:
# test wrangle
df, train_exp, X_train_scaled, y_train, X_test_scaled, y_test = wrangle.wrangle_data()

Acquire: compiling raw data files...
Acquire: Completed!
Prepare: preparing data files...
Prepare: Completed!
(289, 25) (73, 25)


In [3]:
sa = acquire.get_san_antonio_data()

In [4]:
sa.head()

,zip,population,positive,casesp100000,zipint,activecases,activecaserate,shape_length,shape_area
OBJECTID,,,,,,,,,
1,78002,9061,378,4171.724975,78002,24,264.871427,0.427542,0.009546
2,78006,5243,96,1831.012779,78006,7,133.511348,0.552725,0.005416
4,78015,12254,192,1566.835319,78015,16,130.569610,0.278955,0.002312
5,78023,29569,642,2171.192803,78023,49,165.714092,0.886455,0.017922
7,78052,699,28,4005.722461,78052,1,143.061516,0.260085,0.001147


In [5]:
sasvi = acquire.get_sa_svi_data()

In [6]:
sasvi.head()

,st,state,st_abbr,stcnty,county,tract,location,area_sqmi,e_totpop,m_totpop,...,f_crowd,f_noveh,f_groupq,f_theme4,f_total,e_uninsur,m_uninsur,ep_uninsur,mp_uninsur,e_daypop
0,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,0,1,1,3,3,272,130,11.3,5.0,38328
1,48,TEXAS,TX,48029,Bexar,48029110300,"Census Tract 1103, Bexar County, Texas",0.598444,3023,309,...,0,1,0,2,4,606,206,20.0,5.7,9202
2,48,TEXAS,TX,48029,Bexar,48029110500,"Census Tract 1105, Bexar County, Texas",0.448810,2388,243,...,0,1,0,1,9,465,125,19.5,4.4,2044
3,48,TEXAS,TX,48029,Bexar,48029110600,"Census Tract 1106, Bexar County, Texas",0.758136,5301,486,...,0,1,1,3,8,690,265,24.5,7.4,6776
4,48,TEXAS,TX,48029,Bexar,48029110700,"Census Tract 1107, Bexar County, Texas",0.379005,1114,208,...,0,1,0,1,3,251,128,22.5,8.8,3716


In [7]:
huddf = acquire.get_HUD(sasvi)

AttributeError: 'DataFrame' object has no attribute 'zip'

In [ ]:
huddf.head()

In [ ]:
huddf.shape

In [ ]:
merge_bexar = sa[['zip', 'population', 'positive', 'casesp100000']]

In [ ]:
svi_zip = pd.merge(sasvi, huddf, on='tract', how='left')

In [ ]:
svi_zip.isna().sum()


In [ ]:
svi_zip.shape

In [ ]:
svi_zip_cases = pd.merge(svi_zip, merge_bexar, on='zip', how='left')

In [ ]:
svi_zip_cases.head()

In [ ]:
svi_zip_cases.isna().sum()

In [ ]:
svi_zip_cases.shape

In [ ]:
# testing
df = acquire.run()
df = prepare.prepare_data(df)
df.head()

In [ ]:
df.columns

In [ ]:
# full acquire test
df = acquire.run()

In [ ]:
# test acquire import Bexar data
bexar = acquire.get_san_antonio_data()
bexar.head()

In [ ]:
# test acquire import svi data
svi = acquire.get_svi_data()
svi.head()

In [ ]:
# test acquire get HUD merge df
merge_HUD = acquire.get_HUD(bexar)

In [ ]:
merge_HUD.head()

In [ ]:
# import Bexar cases dataframe
bexar_cases = pd.read_csv('Bexar_County_COVID-19_Data_by_Zip_Code.csv', index_col=0)
bexar_cases.columns = bexar_cases.columns.str.lower()
bexar_cases.head()

In [ ]:
bexar_cases.info()

In [ ]:
# how many unique zip codes are in the cases df?
bexar_cases.nunique()

In [ ]:
# import san antonio svi index dataframe
svidf = pd.read_csv('san_antonio_2018_tract.csv')
svidf.columns = svidf.columns.str.lower()
svidf = svidf.rename(columns={'fips': 'tract'})
svidf.head()

In [ ]:
# how many unique FIPS ids are in the svi df?
svidf.tract.value_counts()

In [ ]:
# get list of unique zip codes in the bexar cases df
bexar_zip = bexar_cases.zip_code.tolist()

In [ ]:
# import track to zip dataframe
zips = pd.read_csv('TRACT_ZIP_122018_78s_only.csv')
# filter the zips df to only those in the bexar zip list
zips = zips[zips.zip.isin(bexar_zip)]
zips

In [ ]:
# view zips sorted by tract and total ratio
zips.sort_values(by=['tract', 'tot_ratio'], ascending=[True, False]).head()

In [ ]:
# verify that .08 is highest for tract 48493000201
zips[zips.tract == 48493000201]

In [ ]:
# aggregate the data frame to get the zip code with the max ratio by tract
zipsdf = zips.groupby(['tract'])['tot_ratio', 'zip'].agg({'tot_ratio':['max'], 'zip':['first']})
zipsdf.columns = [' '.join(col).strip() for col in zipsdf.columns.values]
zipsdf = zipsdf.reset_index()

In [ ]:
zipsdf.head()

In [ ]:
# the svi only has 366 rows, hopefully they are all in this list
len(zipsdf)

In [ ]:
# create merge df to test merging
merge_zip3 = zipsdf[['tract', 'zip first']]

In [ ]:
merge_zip3 = merge_zip3.rename(columns={'zip first':'zip'})

In [ ]:
merge_zip3

In [ ]:
# create new df merging svi and 2nd merge zip file on tract
svi_zip2 = pd.merge(svidf, merge_zip3, on='tract', how='left')
svi_zip2.shape

In [ ]:
# check for nulls
svi_zip2.isna().sum()

In [ ]:
# create dataframe to merge with svi
merge_bexar = bexar[['zip', 'population', 'positive', 'casesp100000']]
merge_bexar.head()

In [ ]:
# merge svi_zip with merge_bexar
full = pd.merge(svi_zip2, merge_bexar, on='zip', how='left')
full.head()

In [ ]:
max_ratio = zips.groupby('tract')[['tot_ratio']].max()

In [ ]:
max_ratio = max_ratio.reset_index()

In [ ]:
max_ratio.head()

In [ ]:
max_ratio.nunique()

In [ ]:
zips[zips.zip == 78219]

In [ ]:
# requiring the total ratio to be 54% or higher per track gives the closest to 366 without going over
# there will be 4 zip codes not represented and 2 census tracks not represented
# when merged we should get null values for these
zip_54 = zips[zips.tot_ratio >= .54]
zip_54.nunique()

In [ ]:
# create df with only values to merge
merge_zip = zip_54[['tract', 'zip']]
merge_zip

In [ ]:
# create new df merging svi and merge zip on tract
fulldf = pd.merge(svidf, merge_zip, on='tract', how='left')
fulldf.shape

In [ ]:
# will need to convert zip to int
type(fulldf.zip[0])

In [ ]:
# there are 21 tract ids that do not have a zip code
fulldf.isna().sum()

In [ ]:
# create a df of just the row with a null zip code
missing_zip = fulldf[fulldf.zip.isna()]

In [ ]:
missing_zip

In [ ]:
# here is a list of the tracts missing zip codes
missing_zip.tract.value_counts()

In [ ]:
# find value for 48029131802
zips[zips.tract == 48029131802]
# assign 78101 to this track = highest % addresses

In [ ]:
# find value for 48029182103
zips[zips.tract == 48029182103]
# assign 78023 to this track = highest % addresses

In [ ]:
# find value for 48029980004
zips[zips.tract == 48029980004]
# assign 78216 to this track = highest % addresses

In [ ]:
# find value for 48029130402
zips[zips.tract == 48029130402]
# assign 78210 to this track = highest % addresses

In [ ]:
# find value for 48029141900
zips[zips.tract == 48029141900]
# assign 78263 to this track = highest % addresses

In [ ]:
# find value for 48029191503
zips[zips.tract == 48029191503]
# assign 78249 to this track = highest % addresses

In [ ]:
# find value for 48029181704
zips[zips.tract == 48029181704]
# assign 78238 to this track = highest % addresses

In [ ]:
# find value for 48029131700
zips[zips.tract == 48029131700]
# assign 78148 to this track = highest % addresses

In [ ]:
# find value for 48029121205
zips[zips.tract == 48029121205]
# assign 78233 to this track = highest % addresses

In [ ]:
# find value for 48029110100
zips[zips.tract == 48029110100]
# assign 78205 to this track = highest % addresses

In [ ]:
# find value for 48029182102
zips[zips.tract == 48029182102]
# assign 78255 to this track = highest % addresses

In [ ]:
# find value for 48029192100
zips[zips.tract == 48029192100]
# assign 78204 to this track = highest % addresses

In [ ]:
# find value for 48029121905
zips[zips.tract == 48029121905]
# assign 78259 to this track = highest % addresses

In [ ]:
# find value for 48029130800
zips[zips.tract == 48029130800]
# assign 78219 to this track = highest % addresses

In [ ]:
# find value for 48029121203
zips[zips.tract == 48029121203]
# assign 78233 to this track = highest % addresses

In [ ]:
# find value for 48029171602
zips[zips.tract == 48029171602]
# assign 78227 to this track = highest % addresses

In [ ]:
# find value for 48029181725
zips[zips.tract == 48029181725]
# assign 78251 to this track = highest % addresses

In [ ]:
# find value for 48029150300
zips[zips.tract == 48029150300]
# assign 78214 to this track = highest % addresses

In [ ]:
# find value for 48029191504
zips[zips.tract == 48029191504]
# assign 78231 to this track = highest % addresses

In [ ]:
# find value for 48029191004
zips[zips.tract == 48029191004]
# assign 78216 to this track = highest % addresses

In [ ]:
# find value for 48029130500
zips[zips.tract == 48029130500]
# assign 78202 to this track = highest % addresses

In [ ]:
tracts_list = missing_zip.tract.tolist()

In [ ]:
assign_zip_list = [78101, 78023, 78216, 78210, 78263, 78249, 78238, 78148, 78233, 78205, 78255, 78204, 78259,
                   78219, 78233, 78227, 78251, 78214, 78231, 78216, 78202]

In [ ]:
len(tracts_list)

In [ ]:
len(assign_zip_list)

In [ ]:
# verified lists are same length, now zip lists together into df to merge back to fulldf
# https://cmdlinetips.com/2018/01/how-to-create-pandas-dataframe-from-multiple-lists/#:~:text=Create%20pandas%20dataframe%20from%20lists%20using%20zip%20Second,it%20to%20produce%20one%20item%20at%20a%20time.
data_to_add = pd.DataFrame()
data_to_add['tract'] = tracts_list
data_to_add['zip'] = assign_zip_list
data_to_add

In [ ]:
# add these values to the merge zip df
merge_zip2 = merge_zip.append(data_to_add)

In [ ]:
len(merge_zip2)

In [ ]:
# create new df merging svi and 2nd merge zip file on tract
svi_zip = pd.merge(svidf, merge_zip2, on='tract', how='left')
svi_zip.shape

In [ ]:
# check for nulls
svi_zip.isna().sum()

In [ ]:
svi_zip.head()

In [ ]:
bexar_cases.head()

In [ ]:
# rename the zip code column to merge on
bexar_cases = bexar_cases.rename(columns={'zip_code': 'zip', 'populationtotals_totpop_cy': 'population'})

In [ ]:
bexar_cases

In [ ]:
# create dataframe to merge with svi
merge_bexar = bexar_cases[['zip', 'population', 'positive', 'casesp100000']]
merge_bexar.head()

In [ ]:
# merge svi_zip with merge_bexar
svi_zip_cases = pd.merge(svi_zip, merge_bexar, on='zip', how='left')
svi_zip_cases.head()

In [ ]:
# check for nulls
svi_zip_cases.isna().sum()

In [ ]:
# write final compliation to csv
#svi_zip_cases.to_csv('full_san_antonio.csv')